In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input,MaxPool2D, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate

# 1) Exploring the Dataset

In [2]:
file_path = "Data"
path, dirs, files = next(os.walk(file_path))
print("all data",len(files))

all data 5


In [3]:
data_Dec = pd.read_csv("Data\\2019-Dec.csv")

In [4]:
data_Dec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [5]:
data_Dec.describe()

,product_id,category_id,price,user_id
count,3.533286e+06,3.533286e+06,3.533286e+06,3.533286e+06
mean,5.473054e+06,1.555023e+18,8.871856e+00,5.223318e+08
std,1.331331e+06,1.689262e+17,1.986474e+01,8.494819e+07
min,3.752000e+03,1.487580e+18,-7.937000e+01,1.180452e+06
25%,5.726191e+06,1.487580e+18,2.060000e+00,4.866830e+08
50%,5.811429e+06,1.487580e+18,4.210000e+00,5.566496e+08
75%,5.859462e+06,1.487580e+18,7.140000e+00,5.828019e+08
max,5.917178e+06,2.235524e+18,3.277800e+02,5.954145e+08


In [6]:
data_Dec.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


Property	Description

event_time	Time when event happened at (in UTC).

event_type	Only one kind of event: purchase.

product_id	ID of a product

category_id	Product's category ID

category_code	Product's category taxonomy (code name) if it was possible to make it. Usually present for meaningful 

categories and skipped for different kinds of accessories.

brand	Downcased string of brand name. Can be missed.

price	Float price of a product. Present.

user_id	Permanent user ID.

** user_session**	Temporary user's session ID. Same for each user's session. Is changed every time user come back to online store from a long pause.

view - a user viewed a product

cart - a user added a product to shopping cart

remove_from_cart - a user removed a product from shopping cart

purchase - a user purchased a product

In [7]:
data_Dec[["brand"]].value_counts()

brand    
runail       256217
grattol      155920
irisk        152518
masura       125185
bpw.style     81394
              ...  
ikoo              3
shifei            1
dessata           1
macadamia         1
ibd               1
Length: 252, dtype: int64

In [8]:
data_Dec["event_type"].value_counts()

view                1728331
cart                 927124
remove_from_cart     664655
purchase             213176
Name: event_type, dtype: int64

In [9]:
data_Dec["category_code"].value_counts()

appliances.environment.vacuum             25818
stationery.cartrige                        9500
apparel.glove                              8295
accessories.bag                            4938
furniture.living_room.cabinet              4478
furniture.bathroom.bath                    3529
appliances.personal.hair_cutter            1102
accessories.cosmetic_bag                    693
appliances.environment.air_conditioner       92
furniture.living_room.chair                  20
Name: category_code, dtype: int64

In [10]:
data_Dec["category_id"].value_counts()

1487580007675986893    216293
1487580005595612013    128438
1487580005092295511    123455
1602943681873052386    117336
1487580006317032337    104204
                        ...  
1487580006056985476         1
1487580013363462335         1
1487580011140481125         1
1487580012851757223         1
1933472286753424063         1
Name: category_id, Length: 482, dtype: int64

In [11]:
data_Dec["user_id"].value_counts()

527021202    8689
572631412    3883
467810091    3147
548810441    2426
584289031    2356
             ... 
561547151       1
586485554       1
586486068       1
363648439       1
595414541       1
Name: user_id, Length: 370154, dtype: int64

370154 tane kullanıcı bu siteye ulasmis.İlk defa giren kullanıcının herhangi biriyle benzerliğini hesaplayamadığımız için öncelikle bu siteyi sık sık ziyaret eden bir kişinin diğer kullanıcılar ile benzerliğini hesaplamamız gerekiyor. Datamız event tabanlı olduğu için event türlerine göre kullanıcıların benzerliğini hesaplamamız gerekiyor. 4 tane event type'a sahibiz. 

view -> kullanıcının bir ürünü görüntülemesi

cart -> kullanıcının bir ürünü sepete eklemesi

remove_from_cart-> kullanıcının bir ürünü sepetten çıkarması

purchase -> kullanıcının bu ürünü satın alıp alması

In [12]:
data_Dec[ (data_Dec['user_id'] == 527021202) ] 

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
31413,2019-12-01 10:06:05 UTC,cart,5908242,1487580007675986893,NaN,NaN,1.43,527021202,48a088e0-dafc-4029-87cb-64e0ada013e3
31416,2019-12-01 10:06:07 UTC,cart,5908241,1487580007675986893,NaN,NaN,1.43,527021202,48a088e0-dafc-4029-87cb-64e0ada013e3
31421,2019-12-01 10:06:09 UTC,cart,5908238,1487580007675986893,NaN,NaN,1.43,527021202,48a088e0-dafc-4029-87cb-64e0ada013e3
31423,2019-12-01 10:06:10 UTC,cart,5908234,1487580007675986893,NaN,NaN,1.43,527021202,48a088e0-dafc-4029-87cb-64e0ada013e3
31424,2019-12-01 10:06:12 UTC,cart,5908237,1487580007675986893,NaN,NaN,1.43,527021202,48a088e0-dafc-4029-87cb-64e0ada013e3
...,...,...,...,...,...,...,...,...,...
3156582,2019-12-26 13:23:33 UTC,remove_from_cart,5911236,1487580007675986893,NaN,dartnails,1.43,527021202,594bd193-6326-49a7-b464-e15fb03ecbb9
3156587,2019-12-26 13:23:35 UTC,remove_from_cart,5911853,1487580007675986893,NaN,dartnails,1.27,527021202,594bd193-6326-49a7-b464-e15fb03ecbb9
3156588,2019-12-26 13:23:35 UTC,remove_from_cart,5911864,1487580007675986893,NaN,dartnails,1.27,527021202,594bd193-6326-49a7-b464-e15fb03ecbb9
3156590,2019-12-26 13:23:36 UTC,remove_from_cart,5911857,1487580007675986893,NaN,dartnails,1.27,527021202,594bd193-6326-49a7-b464-e15fb03ecbb9


In [13]:
data_most=data_Dec[ (data_Dec['user_id'] == 527021202) ]
data_most["product_id"].value_counts() 

5911864    55
5911859    55
5911247    53
5911237    51
5911855    48
           ..
5862284     1
5848407     1
5848338     1
5833319     1
5862310     1
Name: product_id, Length: 859, dtype: int64

In [14]:
data_Dec[ (data_Dec['user_id'] == 527021202)& (data_Dec['product_id'] == 5911855)  ]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
755853,2019-12-06 18:21:42 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,9cfb719a-91af-4bf3-84c2-8ace7abf6fc7
755957,2019-12-06 18:22:16 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,9cfb719a-91af-4bf3-84c2-8ace7abf6fc7
755960,2019-12-06 18:22:17 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,9cfb719a-91af-4bf3-84c2-8ace7abf6fc7
756052,2019-12-06 18:22:41 UTC,remove_from_cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,9cfb719a-91af-4bf3-84c2-8ace7abf6fc7
820725,2019-12-07 11:13:42 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,69f34e3a-8fb9-4c9f-9011-664f190065e3
821128,2019-12-07 11:17:53 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,fd2aae2d-64b4-4ccd-8ec5-d0bb0139f40d
821131,2019-12-07 11:17:54 UTC,cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,fd2aae2d-64b4-4ccd-8ec5-d0bb0139f40d
821210,2019-12-07 11:18:29 UTC,remove_from_cart,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,fd2aae2d-64b4-4ccd-8ec5-d0bb0139f40d
945448,2019-12-08 13:07:50 UTC,view,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,b20e458a-7ab7-49ff-ac64-2c8ca35e9ee7
945503,2019-12-08 13:08:20 UTC,view,5911855,1487580007675986893,NaN,dartnails,1.27,527021202,b20e458a-7ab7-49ff-ac64-2c8ca35e9ee7


In [15]:
data_Dec[ (data_Dec['user_id'] == 572631412)& (data_Dec['event_type'] == "purchase") ]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1890162,2019-12-15 13:58:40 UTC,purchase,40094,1487580011383750769,NaN,swarovski,2.06,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890163,2019-12-15 13:58:40 UTC,purchase,40100,1487580011383750769,NaN,swarovski,2.06,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890164,2019-12-15 13:58:40 UTC,purchase,40104,1487580011383750769,NaN,swarovski,2.06,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890165,2019-12-15 13:58:40 UTC,purchase,42615,1487580011383750769,NaN,swarovski,2.06,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890166,2019-12-15 13:58:40 UTC,purchase,42675,1487580011383750769,NaN,swarovski,2.38,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
...,...,...,...,...,...,...,...,...,...
1890356,2019-12-15 13:58:40 UTC,purchase,59892,1487580007634043851,NaN,runail,0.95,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890357,2019-12-15 13:58:40 UTC,purchase,59893,1487580007634043851,NaN,runail,0.95,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890358,2019-12-15 13:58:40 UTC,purchase,59896,1487580007634043851,NaN,runail,0.95,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da
1890359,2019-12-15 13:58:40 UTC,purchase,6192,1487580007659209676,NaN,runail,0.83,572631412,a133acd7-8cf6-e069-e38d-303c4104c3da


In [16]:
data_Dec[ (data_Dec['user_id'] == 572631412)& (data_Dec['product_id'] == 40094) ]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
933875,2019-12-08 11:32:47 UTC,cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,3c50bbeb-0da3-3986-d2b5-694e47c3a15a
934862,2019-12-08 11:41:12 UTC,cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,3c50bbeb-0da3-3986-d2b5-694e47c3a15a
935288,2019-12-08 11:44:16 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,3c50bbeb-0da3-3986-d2b5-694e47c3a15a
940252,2019-12-08 12:21:56 UTC,cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,3c50bbeb-0da3-3986-d2b5-694e47c3a15a
940367,2019-12-08 12:22:53 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,3c50bbeb-0da3-3986-d2b5-694e47c3a15a
1639758,2019-12-13 09:42:05 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,9785cfc3-9783-4f7e-91b3-9cbbe5257fd6
1639760,2019-12-13 09:42:05 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,9785cfc3-9783-4f7e-91b3-9cbbe5257fd6
1639770,2019-12-13 09:42:06 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,9785cfc3-9783-4f7e-91b3-9cbbe5257fd6
1639772,2019-12-13 09:42:06 UTC,remove_from_cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,9785cfc3-9783-4f7e-91b3-9cbbe5257fd6
1717786,2019-12-13 21:29:11 UTC,cart,40094,1487580011383750769,NaN,swarovski,2.06,572631412,d53d1c2e-9195-6051-cf7b-7332dfeae818


In [17]:
data_Dec[ (data_Dec['user_id'] == 527021202)& (data_Dec['product_id'] == 5911855) & (data_Dec['event_type'] == "cart") ].iloc[:,1].count()

34

In [18]:
data_Dec[ (data_Dec['user_id'] == 527021202)& (data_Dec['product_id'] == 5911855) & (data_Dec['event_type'] == "remove_from_cart") ].iloc[:,1].count()

11

# Getting all products


In [19]:
all_product_with_dup=data_Dec[ ['product_id'] ]
all_product_with_dup

,product_id
0,5712790
1,5764655
2,4958
3,5848413
4,5824148
...,...
3533281,5683350
3533282,5888097
3533283,59975
3533284,5775982


In [20]:
all_product=all_product_with_dup.drop_duplicates()
all_product

,product_id
0,5712790
1,5764655
2,4958
3,5848413
4,5824148
...,...
3525123,5692769
3526646,5834911
3526805,5691801
3527734,5838169


In [21]:
all_product.iloc[:,0]

0          5712790
1          5764655
2             4958
3          5848413
4          5824148
            ...   
3525123    5692769
3526646    5834911
3526805    5691801
3527734    5838169
3532733    5916393
Name: product_id, Length: 44624, dtype: int64

# Calculating interest based on events for spesific user and all product 

In [22]:
user_572631412=data_Dec[ data_Dec['user_id'] == 572631412 ]
user_572631412=user_572631412[["event_type","product_id"]]
user_572631412

,event_type,product_id
345939,view,5910166
346132,view,5560756
357509,view,5754148
357562,view,5694630
357841,cart,5694631
...,...,...
1890356,purchase,59892
1890357,purchase,59893
1890358,purchase,59896
1890359,purchase,6192


In [23]:
user_572631412[user_572631412["product_id"] ==5754148]

,event_type,product_id
357509,view,5754148
444206,view,5754148
1773976,cart,5754148
1774643,cart,5754148
1890235,purchase,5754148


In [24]:
def check_if_it_is_purchased(i):
    return user_572631412[user_572631412["product_id"] ==i].sort_index(ascending=False).iloc[0,0]=="purchase"
 

In [25]:
check_if_it_is_purchased(5754148)

True

In [26]:
def calculate_viewing_score(i):
    return user_572631412[user_572631412["product_id"] ==i].groupby(by="event_type").count().tail(1).iloc[0,0] / user_572631412.groupby(by="event_type").count().tail(1).iloc[0,0]

In [27]:
calculate_viewing_score(5754148)

0.05128205128205128

In [28]:
def get_last_event(i):
    droped_viewing=user_572631412[(user_572631412["product_id"] ==i ) & (user_572631412["event_type"] != "view" )]
    return droped_viewing.sort_index(ascending=False).iloc[0,0]
    

In [29]:
def if_it_is_empty(i):
    return len(user_572631412[user_572631412["product_id"] ==i]) == 0

In [30]:
if_it_is_empty(5712790)

True

In [31]:
get_last_event(5884578)

'remove_from_cart'

In [32]:
def if_has_only_view(i):
    return len(user_572631412[user_572631412["product_id"] ==i]) == len(user_572631412[(user_572631412["product_id"] ==i ) & (user_572631412["event_type"] == "view" )])

In [33]:
product_interest=[]
for i in all_product.iloc[:,0]:
    if if_it_is_empty(i):
        product_interest.append(-1)
    elif check_if_it_is_purchased(i):
        product_interest.append(4+calculate_viewing_score(i))
    elif if_has_only_view(i):
        product_interest.append(calculate_viewing_score(i))
    else:
        print(i)
        last_event=get_last_event(i)
        if last_event== "cart":
            product_interest.append(3+calculate_viewing_score(i))
        else:
            product_interest.append(2+calculate_viewing_score(i))

4958
5706778
5810157
5862313
5693501
5833325
5833326
5884578
5848901
5906118
5908186
5819125
5722502
5799881
5908214
5866174
5747357
5862943
5885380
5866158
5862274
5907878
5907874
5859336
4640
4554
4689
5877603
5833332
5843567
5904330
5809261
5833327
5785144
5858911
5877597
5870867
5859327
5866164
5877610
5889907
5904500
4905
5904498
5904802
5787135
5763416
5853627
5568210
5773606
5749145
5885376
5744419
5859081
5809288
5809295
30194
82253
5852385
5834733
5850311
5622678
5747366
5762501
5842740
5833334
5592510
5835869
5833335
5802650
5843545
5781044
5831616
5830318
5809323
5814066
5864736
5864737
5649238
4591
5866173
5780030
5859147
5877489
5749149
5749154
5877599
5809267
5758961
5868465
5599203
5898244
5906117
5757290
5877311
5877328
5809314
5697328
5879280
40090
5809300
5722999
5835856
5570258
5861333
5809303
5833319
5859329
5906109
5885378
5864626
5723270
5809227
5818481
5848909
5847854
40099
5725890
5877738
5807859
5711056
5812957
47612
5707753
5751215
5906122
5867083
5751220
5751

In [34]:
for j in product_interest:
    print(j)

-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.076923076923077
-1
4.0256410256410255
2.3333333333333335
4.0256410256410255
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
2.0256410256410255
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
0.02564102564102564
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0.02564102564102564
-1
-1
-1
-1
-1
-1
-1
3.4615384615384617
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
2.0256410256410255
2.1025641025641026
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
2.076923076923077
-1
-1
-1
2.051282051282051
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
2.051282051282051
-1
-1
2.128205128205128
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
2.1025641025641026
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-

-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
2.128205128205128
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.1025641025641026
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.153846153846154
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
2.2051282051282053
-1
2.0256410256410255
-1
-1
2.076923076923077
-1
-1
-1
-1

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.128205128205128
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


...

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.076923076923077
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4.0256410256410255
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2.076923076923077
-1
2.051282051282051
2.051282051282051
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

# Calculating interest based on events for all user and product

In [35]:
all_users_with_dup=data_Dec[ ['user_id'] ]
all_users_with_dup

,user_id
0,576802932
1,412120092
2,494077766
3,348405118
4,576005683
...,...
3533281,536812729
3533282,503658154
3533283,595414541
3533284,397780878


In [36]:
all_users=all_users_with_dup.drop_duplicates()
all_users

,user_id
0,576802932
1,412120092
2,494077766
3,348405118
4,576005683
...,...
3533262,566918932
3533267,595414257
3533270,571581564
3533282,503658154


In [37]:
def check_if_it_is_purchased(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return data[data["product_id"] ==j].sort_index(ascending=False).iloc[0,0]=="purchase"

In [38]:
def calculate_viewing_score(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return data[data["product_id"] ==j].groupby(by="event_type").count().tail(1).iloc[0,0] / data.groupby(by="event_type").count().tail(1).iloc[0,0]

In [39]:
def get_last_event(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    droped_viewing=data[(data["product_id"] ==j ) & (data["event_type"] != "view" )]
    return droped_viewing.sort_index(ascending=False).iloc[0,0]

In [40]:
def if_it_is_empty(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return len(data[data["product_id"] ==j]) == 0

In [41]:
def if_has_only_view(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return len(data[data["product_id"] ==j]) == len(data[(data["product_id"] ==j ) & (data["event_type"] == "view" )])

In [45]:
product_interests_of_all_users=[]
count=0;
for i in all_users.iloc[:,0]:
    product_interest=[]
    print(count/len(all_users.iloc[:,0] *100))
    for j in all_product.iloc[:,0]:
        if if_it_is_empty(i,j):
            product_interest.append(-1)
        elif check_if_it_is_purchased(i,j):
            product_interest.append(4+calculate_viewing_score(i,j))
        elif if_has_only_view(i,j):
            product_interest.append(calculate_viewing_score(i,j))
        else:
            last_event=get_last_event(i,j)   
            if last_event== "cart":
                product_interest.append(3+calculate_viewing_score(i,j))
            else:
                product_interest.append(2+calculate_viewing_score(i,j))
    product_interests_of_all_users.append(product_interest)        
    count=count+1
    if count==3:
        break

0.0
2.7015782620206725e-06
5.403156524041345e-06


In [46]:
len(product_interests_of_all_users)

3

In [47]:
all_users.head()

,user_id
0,576802932
1,412120092
2,494077766
3,348405118
4,576005683


In [ ]:
df_interest_for_10=pd.DataFrame(product_interests_of_all_users,index=all_users.head(11).iloc[:,0],columns=all_product.iloc[:,0])
df_interest_for_10

In [48]:
from pathlib import Path  


In [49]:
#//filepath = Path('out.csv')  
#//filepath.parent.mkdir(parents=True, exist_ok=True)  
df_interest_for_10.to_csv("out2.csv") 

NameError: name 'df_interest_for_10' is not defined

In [50]:
check = pd.read_csv("out2.csv")

In [51]:
check

,user_id,5712790,5764655,4958,5848413,5824148,5773361,5629988,5807805,5588608,...,44537,5660166,5759999,5878883,5865846,5692769,5834911,5691801,5838169,5916393
0,576802932,2.003484,-1.000000,-1.00,-1.0,-1.00,-1.000000,-1.000000,4.003484,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,412120092,-1.000000,0.033333,-1.00,-1.0,-1.00,-1.000000,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,494077766,-1.000000,-1.000000,5.00,-1.0,-1.00,-1.000000,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,348405118,-1.000000,-1.000000,-1.00,0.5,-1.00,-1.000000,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,576005683,-1.000000,-1.000000,-1.00,-1.0,0.04,-1.000000,-1.000000,2.080000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,560109803,-1.000000,-1.000000,-1.00,-1.0,-1.00,0.052632,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,579966747,-1.000000,-1.000000,-1.00,-1.0,-1.00,-1.000000,4.019608,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,546170008,-1.000000,-1.000000,-1.00,-1.0,-1.00,-1.000000,-1.000000,-1.000000,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,579751441,-1.000000,-1.000000,4.25,-1.0,-1.00,-1.000000,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,579969717,-1.000000,-1.000000,-1.00,-1.0,-1.00,-1.000000,-1.000000,-1.000000,-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [52]:
check.replace(-1,np.NaN)

,user_id,5712790,5764655,4958,5848413,5824148,5773361,5629988,5807805,5588608,...,44537,5660166,5759999,5878883,5865846,5692769,5834911,5691801,5838169,5916393
0,576802932,2.003484,NaN,NaN,NaN,NaN,NaN,NaN,4.003484,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,412120092,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,494077766,NaN,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,348405118,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,576005683,NaN,NaN,NaN,NaN,0.04,NaN,NaN,2.080000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,560109803,NaN,NaN,NaN,NaN,NaN,0.052632,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,579966747,NaN,NaN,NaN,NaN,NaN,NaN,4.019608,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,546170008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,579751441,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,579969717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
data_Dec=data_Dec.drop_duplicates(subset=['event_time'])

In [58]:
def check_if_it_is_purchased(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return len(data[data["product_id"] ==j].loc[data['event_type'].str.contains("purchase")]) > 0 

In [60]:
def calculate_viewing_score(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return data[data["product_id"] ==j].groupby(by="event_type").count().tail(1).iloc[0,0] / data.groupby(by="event_type").count().tail(1).iloc[0,0]

In [61]:
def get_last_event(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    droped_viewing=data[(data["product_id"] ==j ) & (data["event_type"] != "view" )]
    return droped_viewing.sort_index(ascending=False).iloc[0,0]

In [62]:
def if_it_is_empty(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return len(data[data["product_id"] ==j]) == 0

In [63]:
def if_has_only_view(i,j):
    data=data_Dec[ data_Dec['user_id'] == i ]
    data=data[["event_type","product_id"]]
    return len(data[data["product_id"] ==j]) == len(data[(data["product_id"] ==j ) & (data["event_type"] == "view" )])

In [ ]:
# make it better
product_interests_of_all_users=[]
count=0;
for i in all_users.iloc[:,0]:
    product_interest=[]
    print(count/len(all_users.iloc[:,0] *100))
    for j in all_product.iloc[:,0]:
        if if_it_is_empty(i,j):
            product_interest.append(-1)
        elif check_if_it_is_purchased(i,j):
            product_interest.append(4+calculate_viewing_score(i,j))
        elif if_has_only_view(i,j):
            product_interest.append(calculate_viewing_score(i,j))
        else:
            last_event=get_last_event(i,j)   
            if last_event== "cart":
                product_interest.append(3+calculate_viewing_score(i,j))
            else:
                product_interest.append(2+calculate_viewing_score(i,j))
    product_interests_of_all_users.append(product_interest)        
    count=count+1
    if count==11:
        break

In [59]:
check_if_it_is_purchased(572631412,5754148)

True

In [67]:
all_product.reset_index()

,index,product_id
0,0,5712790
1,1,5764655
2,2,4958
3,3,5848413
4,4,5824148
...,...,...
44619,3525123,5692769
44620,3526646,5834911
44621,3526805,5691801
44622,3527734,5838169


In [78]:
data_user_and_product=data_Dec[["user_id","product_id"]].reset_index().drop(columns=['index'])
data_user_and_product

,user_id,product_id
0,576802932,5712790
1,494077766,4958
2,348405118,5848413
3,576005683,5824148
4,560109803,5773361
...,...,...
1654766,536812729,5683350
1654767,503658154,5888097
1654768,595414541,59975
1654769,397780878,5775982


In [97]:
user_ids_with_index=data_user_and_product.drop_duplicates(subset=['user_id']).reset_index().drop(columns=["index","product_id"]).reset_index()
user_ids_with_index

,index,user_id
0,0,576802932
1,1,494077766
2,2,348405118
3,3,576005683
4,4,560109803
...,...,...
219181,219181,566918932
219182,219182,595414257
219183,219183,571581564
219184,219184,503658154


In [93]:
product_ids_with_index=data_user_and_product.drop_duplicates(subset=['product_id']).reset_index().drop(columns=["index","user_id"]).reset_index()
product_ids_with_index

,index,product_id
0,0,5712790
1,1,4958
2,2,5848413
3,3,5824148
4,4,5773361
...,...,...
41588,41588,5876401
41589,41589,5807547
41590,41590,5906533
41591,41591,5916393


In [94]:
product_ids_with_index[product_ids_with_index["product_id"]==5848413].iloc[0,0]

2

In [83]:
data_user_and_product.drop_duplicates(subset=['user_id'])

,user_id,product_id
0,576802932,5712790
1,494077766,4958
2,348405118,5848413
3,576005683,5824148
4,560109803,5773361
...,...,...
1654748,566918932,5650440
1654753,595414257,5812934
1654755,571581564,5858913
1654767,503658154,5888097


In [85]:
all_product=data_user_and_product["product_id"].drop_duplicates()
all_product

0          5712790
1             4958
2          5848413
3          5824148
4          5773361
            ...   
1653251    5876401
1653544    5807547
1653921    5906533
1654287    5916393
1654306    5883269
Name: product_id, Length: 41593, dtype: int64

In [87]:
all_users=data_user_and_product["user_id"].drop_duplicates()
all_users

0          576802932
1          494077766
2          348405118
3          576005683
4          560109803
             ...    
1654748    566918932
1654753    595414257
1654755    571581564
1654767    503658154
1654768    595414541
Name: user_id, Length: 219186, dtype: int64

In [96]:
ones_array = np.ones( (2, 5))*-1
ones_array

array([[-1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.]])

In [99]:
data_user_and_product[data_user_and_product["user_id"]==494077766].drop_duplicates(subset="product_id")["product_id"]

1         4958
8         5335
15     5761411
23     5666749
30     5810157
50     5826809
59     5742717
140    5747361
157       6537
163    5747357
172    5882781
179    5865799
187    5840113
192    5649107
199    5649154
204    5801327
215    5808300
223    5801308
245    5794067
254    5794066
262    5794058
272    5714919
279    5714912
291    5833325
296    5833326
312    5877603
324    5833332
334    5833322
363    5896457
373    5809261
384    5877597
387    5870867
427    5845196
511    5763416
Name: product_id, dtype: int64

In [104]:
all_users

0          576802932
1          494077766
2          348405118
3          576005683
4          560109803
             ...    
1654748    566918932
1654753    595414257
1654755    571581564
1654767    503658154
1654768    595414541
Name: user_id, Length: 219186, dtype: int64

In [126]:
import time


In [133]:
t0= time.time()
product_interests_of_all_users=np.ones( (21910, 41593))*-1
count=0
for i in all_users :
    product_of_users=data_user_and_product[data_user_and_product["user_id"]==i].drop_duplicates(subset="product_id")["product_id"]
    current_index_of_user=user_ids_with_index[user_ids_with_index["user_id"]==i].iloc[0,0]
    for j in product_of_users:
        current_index_of_product=product_ids_with_index[product_ids_with_index["product_id"]==j].iloc[0,0]
        if check_if_it_is_purchased(i,j):
            product_interests_of_all_users[current_index_of_user][current_index_of_product]=4+calculate_viewing_score(i,j)
        elif if_has_only_view(i,j):
            product_interests_of_all_users[current_index_of_user][current_index_of_product]=calculate_viewing_score(i,j)
        else:
            last_event=get_last_event(i,j)   
            if last_event== "cart":
                product_interests_of_all_users[current_index_of_user][current_index_of_product]=3+calculate_viewing_score(i,j)
            else:
                product_interests_of_all_users[current_index_of_user][current_index_of_product]=2+calculate_viewing_score(i,j)
    count=count+1
    if count == 21890:
        break
 
t1 = time.time() - t0
print("Time elapsed: ", t1)

Time elapsed:  2309.9311401844025


In [134]:
np.shape(product_interests_of_all_users)

(21910, 41593)

In [135]:
users_21910=[]
count=0
for k in all_users:
    users_21910.append(k)
    count = count + 1
    if count == 21910:
        break

In [136]:
df_interest_for_21910=pd.DataFrame(product_interests_of_all_users,index=users_21910,columns=all_product)
df_interest_for_21910

product_id,5712790,4958,5848413,5824148,5773361,5629988,5807805,5588608,5335,5755170,...,5844336,5838191,5838194,5838169,5837745,5876401,5807547,5906533,5916393,5883269
576802932,2.004808,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
494077766,-1.000000,5.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,4.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
348405118,-1.000000,-1.0,1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
576005683,-1.000000,-1.0,-1.0,0.071429,-1.000000,-1.0,2.142857,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
560109803,-1.000000,-1.0,-1.0,-1.000000,0.071429,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572288821,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
379214244,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
581230190,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
519845416,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [137]:
df_interest_for_21910.replace(-1,np.NaN).reset_index()

product_id,index,5712790,4958,5848413,5824148,5773361,5629988,5807805,5588608,5335,...,5844336,5838191,5838194,5838169,5837745,5876401,5807547,5906533,5916393,5883269
0,576802932,2.004808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,494077766,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348405118,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,576005683,NaN,NaN,NaN,0.071429,NaN,NaN,2.142857,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,560109803,NaN,NaN,NaN,NaN,0.071429,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21905,572288821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21906,379214244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21907,581230190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21908,519845416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
check.replace(-1,np.NaN)

,user_id,5712790,5764655,4958,5848413,5824148,5773361,5629988,5807805,5588608,...,44537,5660166,5759999,5878883,5865846,5692769,5834911,5691801,5838169,5916393
0,576802932,2.003484,NaN,NaN,NaN,NaN,NaN,NaN,4.003484,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,412120092,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,494077766,NaN,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,348405118,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,576005683,NaN,NaN,NaN,NaN,0.04,NaN,NaN,2.080000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,560109803,NaN,NaN,NaN,NaN,NaN,0.052632,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,579966747,NaN,NaN,NaN,NaN,NaN,NaN,4.019608,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,546170008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,579751441,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,579969717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
df_interest_for_21910.replace(-1,np.NaN).reset_index().to_csv('interest_for_21911_users.csv')